In [3]:
import pandas as pd
import os
import itertools
import numpy as np
import time

In [5]:
class Market:
    def __init__(self, currencies, dataPath, referenceCurrencyM='USD', 
                 initialValue=1000, transactionFee=0.005, initialTime=None, timeframe=50):
        self.path = dataPath
        self.currencies = []
        self.value = initialValue
        self.referenceCurrency = referenceCurrencyM
        self.reference = referenceCurrencyM
        self.fee = transactionFee
        self.time = initialTime #Fill in
        self.currencies = (currencies)
        self.currencies.remove(self.reference)
        self.currencies.insert(0, self.reference)
        self.m = len(self.currencies)
        self.majorPairs = ['EURUSD', 'GBPUSD']
        self.portfolio = None
        pairs = list(itertools.permutations(currencies, 2))
        self.df = self.importFile(pairs)
        self.timeframe = timeframe
        self.initPortfolio()
        self.fillInData()
        self.allDates = self.getAllDates()
        
    def import_file(self, pairs):
        df = {}
        dataPath = self.path
        availableFiles = os.listdir(dataPath)
        for pairTuple in pairs:
            pair = pairTuple[0] + pairTuple[1]
            if (pair + '.csv') in availableFiles:
                if dataPath.endswith('/'):
                    if os.path.isfile(dataPath + pair + '.csv'):
                        df[pair] = pd.read_csv(dataPath + pair + '.csv', delimiter='\t', 
                                                usecols=['Timestamp', 'Open', 'High', 'Low', 'Vol'])
                        num = df[pair]._get_numeric_data()
                        num[num < 0] = 1
                    else:
                        continue
                else:
                    if os.path.isfile(dataPath + '/' + pair + '.csv'):
                        df[pair] = (pd.read_csv(dataPath + '/' + pair + '.csv', delimiter='\t', 
                                                usecols=['Timestamp', 'Open', 'High', 'Low', 'Vol']))
                        num = df[pair]._get_numeric_data()
                        num[num < 0] = 1
                    else:
                        continue
        return df
    
    def get_portfolio_size(self):
        return self.m
    
    def get_pairs(self):
        return self.pairs
    
    def init_portfolio(self):
        np.random.seed(5)
        self.portfolio = np.random.rand(len(self.currencies), 1)
        summation = np.sum(self.portfolio)
        self.portfolio = np.divide(self.portfolio, summation)
        self.portfolio = np.round(self.portfolio, 2)
        for i in range(self.timeframe):
            self.timestep()
        
    def reallocate(self, currencyAllocation):
        exchange = np.subtract(currencyAllocation, self.portfolio)
        oldPortfolio = self.portfolio.copy()
        while (exchange == 0.0).all() != True:
            minIndex = np.unravel_index(np.argmin(exchange, axis=None), exchange.shape)
            maxIndex = np.unravel_index(np.argmax(exchange, axis=None), exchange.shape)
            self.exchangeCurrencies(minIndex, maxIndex, min(abs(exchange[minIndex]), exchange[maxIndex]))
            saveValue = exchange[minIndex]
            exchange[minIndex] = exchange[maxIndex] + exchange[minIndex]
            exchange[maxIndex] = saveValue + exchange[maxIndex]
            exchange = np.round(exchange, 2)

    def increment_time(self):
        currentTime = str(self.getCurrentTime())
        currentIndex = self.df[self.majorPairs[0]].loc[self.df[self.majorPairs[0]]['Timestamp'] 
                                                       == currentTime].index.values[0]
        self.time = str(self.df[self.majorPairs[0]]['Timestamp'].loc[currentIndex + 1])
        
    def timestep(self):
        if self.time is None:
            for pair in self.majorPairs:
                if pair in self.df.keys():
                    self.time = self.df[pair]['Timestamp'].loc[0]
        else:
            self.incrementTime()
    
    def get_current_time(self):
        return self.time
    
    def calculate_value(self):
        totalValue = self.value
        priceChange = self.getRates(self.getCurrentTime())
        newValue = np.sum(np.multiply(self.portfolio, priceChange))
        self.value *= newValue
        
    def exchange_currencies(self, source, target, amount):
        self.portfolio[source] -= amount
        self.portfolio[target] += amount
        self.value -= amount * self.fee * self.value
 
    #Needs fixing
    def get_rates(self, dateIndex):
        rates = np.zeros(shape=(len(self.currencies), 1))
        rates[0,0] = 1
        for i in range(1, len(self.currencies)):
            current = self.currencies[i]
            pair = self.referenceCurrency + current
            if current == self.referenceCurrency:
                continue
            if pair in self.df.keys():
                index = dateIndex - 1
                dividers = self.df[pair].iloc[index:index+2]['Open'].values
                initialPrice = dividers[0]
                finalPrice = dividers[1]
                rates[i, 0] = 1/(finalPrice/initialPrice)
            else:
                pair = current + self.referenceCurrency
                index = dateIndex - 1
                dividers = self.df[pair].iloc[index:index+2]['Open'].values
                initialPrice = dividers[0]
                finalPrice = dividers[1]
                rates[i, 0] = finalPrice/initialPrice
        return rates
    
    def fill_in_data(self):
        referenceFrame = self.df[self.majorPairs[0]]
        for pair in self.df.keys():
            if pair != self.majorPairs[0]:
                editFrame = self.df[pair]
                firstDate = editFrame.loc[0, 'Timestamp']
                index = referenceFrame.loc[referenceFrame['Timestamp'] == firstDate].index.values
                i = 1
                while len(index) == 0:
                    index = referenceFrame.loc[referenceFrame['Timestamp'] == editFrame.loc[i, 'Timestamp']].index.values
                    i += 1
                extracted = referenceFrame.iloc[:index[0]].copy()
                if extracted.size == 0:
                    continue
                startValue = editFrame.loc[0, 'Open']
                amplify = (index[0]/250000)
                extracted['Open'] = np.fromfunction(lambda i, j: (1/amplify)*startValue + i*((1-(1/amplify))*startValue/index[0]), shape=(index[0], 1))
                extracted['Vol'] = np.zeros(shape=(index[0], 1))
                extracted['High'] = np.fromfunction(lambda i, j: (1/amplify)*startValue + i*((1-(1/amplify))*startValue/index[0]), shape=(index[0], 1))
                extracted['Low'] = np.fromfunction(lambda i, j: (1/amplify)*startValue + i*((1-(1/amplify))*startValue/index[0]), shape=(index[0], 1))
                self.df[pair] = pd.concat([extracted, self.df[pair]])
                self.df[pair] = self.df[pair].reset_index(drop=True)
                
    def get_all_dates(self):
        return self.df[self.majorPairs[0] if self.majorPairs[0] in self.df.keys() else self.majorPairs[1]].loc[:, 'Timestamp'].values
    
    def process_time_period(self, timePeriod, index, size):
        allPrices = np.zeros(shape=(size, len(self.currencies), timePeriod, 3))
        allRates = np.zeros(shape=(size, len(self.currencies), 1))
        dimensions = ['Open', 'High', 'Low']
        m = 0
        for currency in self.currencies:
            if currency + self.referenceCurrency in self.df.keys():
                pair = currency + self.referenceCurrency
            elif self.referenceCurrency + currency in self.df.keys():
                pair = self.referenceCurrency + currency
            elif self.referenceCurrency == currency:
                for i in range(size):
                    allPrices[i, m, :, :] = 1
                    allRates[i, m, 0] = 1
                m += 1
                continue
            else:
                raise ValueError('Wrong currency parameter.')
            batchValues = self.df[pair].iloc[index : index + timePeriod + size, 1:4].values
            for i in range(size):
                movement = batchValues[i:timePeriod+i]
                refVal = movement[-1][0]
                movement = movement / refVal
                nextPrice = batchValues[timePeriod+i][0]
                rate = nextPrice / refVal
                if (pair[0:3] == 'USD'):
                    movement[0:3] **= -1
                    allPrices[i, m, :, :] = movement
                    allRates[i, m, 0] = 1/rate
                else:
                    allPrices[i, m, :, :] = movement
                    allRates[i, m, 0] = rate
            m += 1
        return (allPrices, allRates)